In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.expand_frame_repr', False)
pd.set_option('display.precision', 2)

In [ ]:
## Here if we want to do any change over time comparison, 2016 vs 2020

# df1 = pd.read_csv('2016.csv')
# print(df1.shape)
# df1.head()

In [31]:
df2 = pd.read_csv('2020.csv')
print(df2.shape)
df2.sample(5)

(3159, 10)


,state_name,county_fips,county_name,votes_gop,votes_dem,total_votes,diff,per_gop,per_dem,per_point_diff
1384,Minnesota,27107,Norman County,1954,1404,3434,550,0.57,0.41,0.16
2927,Virginia,51191,Washington County,21678,6617,28684,15061,0.76,0.23,0.53
2173,Oklahoma,40051,Grady County,18538,4144,23101,14394,0.80,0.18,0.62
3076,Wisconsin,55025,Dane County,78789,260157,344745,-181368,0.23,0.75,-0.53
580,Idaho,16025,Camas County,507,149,754,358,0.67,0.20,0.47


In [33]:
# There are 2 ones exactly tied--dropping those
df2.query('votes_gop != votes_dem', inplace = True)
print(df2.shape)

# Creating class label: GOP win is 0, DEM win is 1
def call_winner(x):
    if x['votes_gop'] > x['votes_dem']:
        return 0
    else:
        return 1

df2['winner'] = df2.apply(call_winner, axis=1)

df2['winner'].value_counts()

(3157, 11)


0    2614
1     543
Name: winner, dtype: int64

In [ ]:
'''
To-do:
- Pull in data from other sources + join
- import scikit-learn and run test models
'''

In [44]:
import censusdata as cd
# These are all of our options for columns to use from the DP05 data profile.
# Note that they include estimates and population estimates, and many dependent variables
# Also see DP02 and DP03
tbl = cd.printtable(cd.censustable('acs5', 2018, 'DP05'))

Variable     | Table                          | Label                                                    | Type 
-------------------------------------------------------------------------------------------------------------------
DP05_0001E   | ACS DEMOGRAPHIC AND HOUSING ES | !! !! Estimate SEX AND AGE Total population              | int  
DP05_0001PE  | ACS DEMOGRAPHIC AND HOUSING ES | !! !! Percent Estimate SEX AND AGE Total population      | int  
DP05_0002E   | ACS DEMOGRAPHIC AND HOUSING ES | !! !! !! Estimate SEX AND AGE Total population Male      | int  
DP05_0002PE  | ACS DEMOGRAPHIC AND HOUSING ES | !! !! !! Percent Estimate SEX AND AGE Total population M | float
DP05_0003E   | ACS DEMOGRAPHIC AND HOUSING ES | !! !! !! Estimate SEX AND AGE Total population Female    | int  
DP05_0003PE  | ACS DEMOGRAPHIC AND HOUSING ES | !! !! !! Percent Estimate SEX AND AGE Total population F | float
DP05_0004E   | ACS DEMOGRAPHIC AND HOUSING ES | !! !! !! Estimate SEX AND AGE Total populatio

In [49]:
# When we decide what we want, we update the variable set in this call, already set to pull all counties in U.S.
# Good options:
# Median age: DP05_0018E
# % under 18 YO: DP05_0019PE
# % over 62 YO: DP05_0023PE
# % one race (white): DP05_0037PE
# % one race (black): DP05_0038PE
# % hispanic: DP05_0071PE
census = (cd.download('acs5', 2018, cd.censusgeo([('county', '*')]),
                                   ['DP05_0018E','DP05_0019PE','DP05_0023PE','DP05_0037PE','DP05_0038PE','DP05_0071PE'],tabletype='profile'))
df = pd.DataFrame(census)
df = df.reset_index()
df = df.rename(columns={"index":"location"})

In [50]:
df.head()

,location,DP05_0018E,DP05_0019PE,DP05_0023PE,DP05_0037PE,DP05_0038PE,DP05_0071PE
0,"Washington County, Mississippi: Summary level:...",36.9,26.2,18.8,25.7,72.2,1.5
1,"Perry County, Mississippi: Summary level: 050,...",40.9,23.4,21.1,78.8,19.7,1.5
2,"Choctaw County, Mississippi: Summary level: 05...",44.0,22.2,25.6,67.7,31.2,0.4
3,"Itawamba County, Mississippi: Summary level: 0...",40.0,21.9,20.5,90.9,7.2,1.5
4,"Carroll County, Mississippi: Summary level: 05...",47.0,19.4,25.8,64.4,34.6,0.3


In [51]:
# should probably turn this into a function if going to need to run on more than 1 table

# censusdata returns an ugly concatenated string with county, state, and FIPS--break this apart
location = df.location.astype(str).str.split(':', expand=True).rename(columns=
                                                                          {0 : 'CountyState', 
                                                                           1 : 'drop1', 
                                                                           2 : 'drop2',
                                                                           3 : 'statefips', 4 : 'ctyfips'})
location.drop(['drop1','drop2'], axis='columns', inplace = True)
location['statefips'] = location['statefips'].astype(str).str[:2]
# after carving out the state and cty fips, concatenate
location['fips'] = location['statefips'] + location['ctyfips'].astype(str)
# For human readability and possibly labels, keep county and state and split up
cs = location.CountyState.astype(str).str.split(',', expand=True).rename(columns=
                                                                          {0 : 'County', 
                                                                           1 : 'State'})
cs['fips'] = location['fips']
cs.sample(5)

,CountyState,drop1,drop2,statefips,ctyfips
0,"Washington County, Mississippi",Summary level,"050, state",28> county,151
1,"Perry County, Mississippi",Summary level,"050, state",28> county,111
2,"Choctaw County, Mississippi",Summary level,"050, state",28> county,019
3,"Itawamba County, Mississippi",Summary level,"050, state",28> county,057
4,"Carroll County, Mississippi",Summary level,"050, state",28> county,015


In [59]:
# Joining cs assumes we will merge on State and County (set multi-index?).
# If we decide to merge on FIPS instead, join with -location- in previous cell
df = df.join(cs, on=None)
df.drop('location', axis = 1, inplace = True)
df.sample(10)

,DP05_0018E,DP05_0019PE,DP05_0023PE,DP05_0037PE,DP05_0038PE,DP05_0071PE,County,State,fips
2833,41.4,22.4,23.2,64.7,0.4,5.1,Swain County,North Carolina,37173
578,35.3,27.1,17.2,92.4,1.7,57.4,San Patricio County,Texas,48409
2028,36.6,25.7,16.1,74.3,16.7,6.4,Columbia County,Georgia,13073
2875,44.5,19.0,24.6,93.2,4.1,0.9,Belmont County,Ohio,39013
2248,38.9,24.5,19.8,96.4,0.6,1.9,Decatur County,Indiana,18031
3192,40.0,25.9,21.1,97.3,0.4,3.0,Jay County,Indiana,18075
654,46.1,18.8,25.0,96.7,0.6,1.4,Orange County,Vermont,50017
1035,36.0,21.2,20.4,60.6,35.0,2.7,Columbia County,Arkansas,05027
2334,52.3,15.4,33.5,86.9,0.5,2.3,Cook County,Minnesota,27031
1740,39.3,21.6,20.3,61.4,28.4,5.3,Baltimore County,Maryland,24005


In [92]:
print(df.shape)
print(df2.shape)

df2.rename(columns = {'fips': 'county_fips'}, inplace = True)
# df.astype({'county_fips': 'int64'}, copy = False)
df['county_fips']=df['county_fips'].astype(int)
# df2.astype({'county_fips': 'int64'}, copy = False)

to_join = df2[['county_fips','winner','per_point_diff']]
# Getting ValueError: You are trying to merge on object and int64 columns. If you wish to proceed you should use pd.concat
# but both columns should be int64 as indicated above...
# could set fips as indices in both and concat, I suppose, but seems unnecessary. Why isn't astype fixing this?
df = df.merge(to_join, on = 'county_fips', how='inner')
print(df.shape)

(3220, 9)
(3157, 11)


In [ ]:
# split into x and y for ML
x_cols = ['DP05_0018E','DP05_0019PE','DP05_0023PE','DP05_0037PE','DP05_0038PE','DP05_0071PE']
X = df[x_cols]
# discrete for classification
y1 = df['winner']
# continuous for win percentage (would be especially interesting to run on 2016 data to predict 2020)
y2 = df['per_point_diff']

# Optional: preprocess percentile data by applying z-scores
# from scipy.stats import zscore
# X2 = X.apply(zscore)

In [101]:
# split the data for a test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y1, test_size=0.2, random_state=10)
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

# fit model and get predictions
from sklearn.naive_bayes import GaussianNB
from sklearn import metrics
clf = GaussianNB()
clf.fit(X_train, y_train)

pred = clf.predict(X_test)
# 6) Use different metrics, such as Accuracy and F1-score, to evaluate your model
print(metrics.classification_report(y_test, pred))

# get confusion matrix
from sklearn.metrics import confusion_matrix
cmx = pd.DataFrame(confusion_matrix(y_test, pred))
print(cmx)

(2488, 6) (2488,) (622, 6) (622,)
              precision    recall  f1-score   support

           0       0.88      0.86      0.87       512
           1       0.42      0.47      0.45       110

    accuracy                           0.79       622
   macro avg       0.65      0.67      0.66       622
weighted avg       0.80      0.79      0.80       622

     0   1
0  441  71
1   58  52


In [102]:
# Testing with a second flavor of NB: https://scikit-learn.org/stable/modules/naive_bayes.html
from sklearn.naive_bayes import ComplementNB
clf = ComplementNB()
clf.fit(X_train, y_train)

pred = clf.predict(X_test)
print(metrics.classification_report(y_test, pred))

cmx = pd.DataFrame(confusion_matrix(y_test, pred))
print(cmx)

              precision    recall  f1-score   support

           0       0.89      0.77      0.83       512
           1       0.35      0.57      0.43       110

    accuracy                           0.73       622
   macro avg       0.62      0.67      0.63       622
weighted avg       0.80      0.73      0.76       622

     0    1
0  393  119
1   47   63


In [ ]:
# Still to test:
# NNC: https://scikit-learn.org/stable/modules/neighbors.html#nearest-neighbors-classification
# Decision Tree: https://scikit-learn.org/stable/modules/tree.html#classification
# get cross validation scores
# interpret issues (with SHAP?)

# Option B: Train on 2016 w/ 2016 census data, pred 2020 data. Then:
# merge y pred and create new column assigning color in choropleth map
# create choropleth map of TP, TN, FP, FN
# reattempt w/ ensemble of Niloofar's methods

### Inspiration:
Classification with Census: https://anaconda.org/muertala/adult-income-census_binary-classification/notebook

### Documentation:
censusdata package: https://jtleider.github.io/censusdata/example1.html